In [1]:
import pandas as pd
import polars as pl
import datetime as dt
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)
import numpy as np

In [11]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [12]:
tb = pd.read_csv('Hammer_10m_18ema_15_50.csv')

In [13]:
tb.tail()

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME
1283,2025-03-21,Friday,2025-03-27,6,23150,P,119.50,09:15:00,130.00,0.10,2025-03-27,15:25:00,40.188518,EOD exit,119.4,1.196,863.930886,102120.086393,10.212009,2025,3,123.25,0.05,10.319654,250.000000,2025-03-21 09:15:00
1284,2025-03-21,Friday,2025-03-27,6,23200,C,123.15,09:15:00,144.85,144.85,2025-03-21,09:25:00,45.522388,SL hit,-21.7,2.680,862.068966,-21017.241379,-2.101724,2025,3,154.90,120.60,0.219828,250.000000,2025-03-21 09:15:00
1285,2025-03-21,Friday,2025-03-27,6,23200,C,192.90,13:45:00,292.85,351.90,2025-03-24,09:15:00,40.838948,Gap SL hit,-159.0,5.448,200.100050,-32906.053027,-3.290605,2025,3,358.40,188.65,0.085043,58.029015,2025-03-21 13:45:00
1286,2025-03-25,Tuesday,2025-03-27,2,23800,C,84.85,12:05:00,190.00,0.05,2025-03-27,15:25:00,38.732230,EOD exit,84.8,0.849,190.204470,15967.855445,1.596786,2025,3,105.90,0.05,1.612934,56.585830,2025-03-25 12:05:00
1287,2025-03-28,Friday,2025-04-03,6,23600,P,110.65,10:25:00,189.95,189.95,2025-03-28,13:45:00,45.078847,SL hit,-79.3,3.006,252.206810,-20758.133670,-2.075813,2025,3,205.35,106.10,0.114754,74.401009,2025-03-28 10:25:00


In [14]:
# tb['Entry Time'] = dt.time(9, 15)

In [30]:
def round_to_nearest_50(value):
    return int(50 * round(value / 50))

def get_hedge_pct(margin):
    if 100 < margin <= 150:
        return 5
    elif 150 < margin <= 200:
        return 4
    elif 200 < margin < 300:
        return 3
    else:
        return 0  # Default value if margin doesn't fit any range

async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    # hedge_pct = 2.75
    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    df["Hedge Strike"] = df.apply(
        lambda row: (
            int(round_to_nearest_50(row["strike"] * (1 + (get_hedge_pct(row["Margin"]) / 100))))
            if row["type"] == "C" and row["Margin"] > 100
            else int(round_to_nearest_50(row["strike"] * (1 - (get_hedge_pct(row["Margin"]) / 100))))
            if row["type"] == "P" and row["Margin"] > 100
            else 0  # Default value if neither condition matches
        ),
        axis=1,
    )

    # df["Hedge Contract"] = df.apply(
    #     lambda row: get_option_contract_name2(
    #         index_name, row["Hedge Strike"], row["Expiry"], row["Option Type"]
    #     ),
    #     axis=1,
    # )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    # print(df['Hedge Strike'].to_string())

    for i in range(0, len(df)):
        print(df.iloc[i]['date'])
        if df.iloc[i]['Hedge Strike']:

            # print(df.iloc[i]["Week"])
            entry_date = df.iloc[i]['date']
            entry_time = df.iloc[i]['Entry Time']
            exit_date = df.iloc[i]['Exit date']
            exit_time = pd.to_datetime(df.iloc[i]['Exit Time']).time()
            hedge_strike = int(df.iloc[i]["Hedge Strike"])
            hedge_expiry = df.iloc[i]["expiry"]
            hedge_opt_type = df.iloc[i]["type"]
            # hedge_entry_datetime = dt.datetime.combine(entry_date, entry_time)
            # hedge_exit_datetime = dt.datetime.combine(exit_date, exit_time)
    
            entry_time_obj = dt.datetime.strptime(entry_time, "%H:%M:%S").time() if isinstance(entry_time, str) else entry_time
            exit_time_obj = dt.datetime.strptime(exit_time, "%H:%M:%S").time() if isinstance(exit_time, str) else exit_time
            
            hedge_df = await fetch_data(
                index=index_str_for_opt,
                start_date=dt.datetime.strptime(entry_date, "%Y-%m-%d").date(),
                end_date=dt.datetime.strptime(exit_date, "%Y-%m-%d").date(),
                start_time=entry_time_obj,
                end_time=exit_time_obj,
                expiry=dt.datetime.strptime(hedge_expiry, "%Y-%m-%d"),
                strike=hedge_strike,
                asset_class=hedge_opt_type[:1],
            )
            if not isinstance(hedge_df, str) and hedge_df is not None:
                hedge_df = hedge_df.to_pandas()
            else:
                print(hedge_df)
                df.loc[i, "Hedge Entry Price"] = 0
                df.loc[i, "Hedge Exit Price"] = 0
                continue
            # print(hedge_df)
    
            hedge_entry_price = hedge_df.iloc[0]["c"]
            hedge_exit_price = hedge_df.iloc[-1]["c"]
    
            print(hedge_entry_price, hedge_exit_price)
    
            df.loc[i, "Hedge Entry Price"] = hedge_entry_price
            df.loc[i, "Hedge Exit Price"] = hedge_exit_price
    
    return df

In [31]:
tb_with_hedge = await add_hedges(tb)
# tb_with_hedge.tail()

2019-01-25
1.25 0.05
2019-01-28
2019-01-30
2019-01-30
0.05 0.05
2019-02-11
1.2 0.35
2019-02-11
2019-02-15
1.2 0.2
2019-02-15
2019-02-20
2019-02-22
3.8 3.35
2019-02-22
8.7 12.4
2019-02-22
3.45 3.3
2019-02-25
1.85 1.45
2019-02-26
0.6 0.75
2019-02-26
2019-02-27
2019-03-01
2.05 0.7
2019-03-05
4.6 0.05
2019-03-06
0.25 0.3
2019-03-08
1.1 1.0
2019-03-08
4.0 0.05
2019-03-11
None
2019-03-12
2019-03-13
0.5 0.25
2019-03-14
1.05 1.8
2019-03-18
0.8 0.05
2019-03-18
3.35 3.45
2019-03-18
1.35 0.05
2019-03-22
3.85 0.05
2019-03-22
5.0 5.8
2019-03-22
4.5 5.45
2019-03-25
4.7 0.05
2019-03-29
10.15 0.05
2019-03-29
5.1 4.5
2019-04-01
0.75 0.05
2019-04-05
2.0 2.05
2019-04-05
4.25 2.8
2019-04-08
2019-04-09
2.3 1.8
2019-04-09
1.45 0.05
2019-04-12
1.7 1.6
2019-04-12
3.5 3.5
2019-04-12
2.65 0.05
2019-04-15
1.1 1.1
2019-04-15
1.0 0.9
2019-04-16
None
2019-04-23
2.25 1.25
2019-04-23
1.55 0.95
2019-04-23
1.15 0.05
2019-04-26
2.85 0.05
2019-04-26
0.95 0.9
2019-04-30
0.25 0.05
2019-05-03
2.2 2.1
2019-05-03
1.2 1.05
201

In [32]:
tb_with_hedge['Hedge Points'] = tb_with_hedge['Hedge Exit Price'] - tb_with_hedge['Hedge Entry Price']
tb_with_hedge['Hedge PnL'] = tb_with_hedge['Hedge Points'] * tb_with_hedge['Qty']
tb_with_hedge['Hedge ROI%'] = tb_with_hedge.apply(
    lambda row: (row['Hedge PnL'] * 100 / 1000000) if not pd.isna(row['Hedge Points']) else 0,
    axis=1
)
tb_with_hedge['Total ROI%'] = tb_with_hedge['ROI%'] + tb_with_hedge['Hedge ROI%']

In [33]:
tb_with_hedge['Total ROI%'].sum()

558.665709092427

In [34]:
tb_with_hedge.tail(50)

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME,Hedge Strike,Hedge Entry Price,Hedge Exit Price,Hedge Points,Hedge PnL,Hedge ROI%,Total ROI%
1238,2025-01-06,Monday,2025-01-09,3,24050,P,171.10,10:05:00,265.65,265.65,2025-01-06,10:35:00,48.073661,SL hit,-94.55,4.3675,211.528292,-20923.849815,-2.092385,2025,1,275.95,152.50,0.393443,63.590693,2025-01-06 10:05:00,0,NaN,NaN,NaN,NaN,0.000000,-2.092385
1239,2025-01-07,Tuesday,2025-01-09,2,23650,P,104.40,12:15:00,176.70,176.70,2025-01-08,12:35:00,41.683831,SL hit,-72.30,2.8110,276.625173,-20777.593361,-2.077759,2025,1,176.85,65.40,1.078838,81.777317,2025-01-07 12:15:00,0,NaN,NaN,NaN,NaN,0.000000,-2.077759
1240,2025-01-08,Wednesday,2025-01-09,1,23500,P,52.90,13:15:00,83.70,0.05,2025-01-09,15:25:00,43.913481,EOD exit,52.85,0.5295,649.350649,33974.350649,3.397435,2025,1,54.55,0.05,3.431818,190.746753,2025-01-08 13:15:00,22550,1.35,0.05,-1.30,-844.155844,-0.084416,3.313019
1241,2025-01-10,Friday,2025-01-16,6,23600,C,169.30,09:15:00,181.25,181.25,2025-01-10,11:15:00,39.551358,SL hit,-11.95,3.5055,847.457627,-13097.881356,-1.309788,2025,1,197.75,102.30,5.677966,250.000000,2025-01-10 09:15:00,24300,9.00,9.65,0.65,550.847458,0.055085,-1.254703
1242,2025-01-10,Friday,2025-01-16,6,23600,C,141.25,11:35:00,197.75,0.05,2025-01-16,15:25:00,42.142412,EOD exit,141.20,1.4130,353.982301,49482.123894,4.948212,2025,1,165.15,0.05,4.998230,104.424779,2025-01-10 11:35:00,24800,1.50,0.05,-1.45,-513.274336,-0.051327,4.896885
1243,2025-01-10,Friday,2025-01-16,6,23550,P,174.15,12:05:00,292.00,348.80,2025-01-13,09:15:00,42.717149,Gap SL hit,-174.65,5.2295,169.707255,-30526.856173,-3.052686,2025,1,392.35,147.25,0.456513,49.957573,2025-01-10 12:05:00,0,NaN,NaN,NaN,NaN,0.000000,-3.052686
1244,2025-01-14,Tuesday,2025-01-16,2,23150,P,71.30,13:05:00,108.50,0.05,2025-01-16,15:25:00,48.523748,EOD exit,71.25,0.7135,537.634409,37922.849462,3.792285,2025,1,82.95,0.05,3.830645,155.577957,2025-01-14 13:05:00,22200,5.55,0.05,-5.50,-2956.989247,-0.295699,3.496586
1245,2025-01-17,Friday,2025-01-23,6,23300,C,138.30,09:15:00,223.65,223.65,2025-01-21,09:15:00,28.647132,SL hit,-85.35,3.6195,234.329233,-20848.154657,-2.084815,2025,1,300.00,103.40,0.817809,68.248389,2025-01-17 09:15:00,0,NaN,NaN,NaN,NaN,0.000000,-2.084815
1246,2025-01-17,Friday,2025-01-23,6,23250,P,194.90,13:55:00,259.90,259.90,2025-01-21,14:35:00,42.841037,SL hit,-65.00,4.5480,307.692308,-21399.384615,-2.139938,2025,1,292.35,57.20,4.236923,89.423077,2025-01-17 13:55:00,0,NaN,NaN,NaN,NaN,0.000000,-2.139938
1247,2025-01-21,Tuesday,2025-01-23,2,23450,C,98.95,09:25:00,125.05,0.05,2025-01-23,15:25:00,41.947029,EOD exit,98.90,0.9900,766.283525,75026.819923,7.502682,2025,1,123.40,0.05,7.578544,224.616858,2025-01-21 09:25:00,24150,2.25,0.05,-2.20,-1685.823755,-0.168582,7.334100


In [38]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["Total ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
        "Total ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0]["Total ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["Total ROI%"].cumsum()
        - combined_df_sorted["Total ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [39]:
stats = generate_stats(tb, 'Hammer W Hedges')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,141.642542,191,39.267016,5.042377,-2.039101,-18.492177,7.659593,Hammer W Hedges
2020,94.3756,221,38.914027,4.809288,-2.364616,-23.056234,4.093279,Hammer W Hedges
2021,91.682089,204,40.686275,4.50444,-2.332119,-14.363032,6.383199,Hammer W Hedges
2022,78.508951,221,35.746606,5.457255,-2.483198,-21.976399,3.572421,Hammer W Hedges
2023,69.00891,187,37.967914,4.564347,-2.198791,-16.131805,4.277817,Hammer W Hedges
2024,56.941168,210,34.285714,5.10179,-2.249186,-25.440187,2.238237,Hammer W Hedges
2025,26.506449,54,40.740741,4.632006,-2.356178,-10.690137,2.479524,Hammer W Hedges
Overall,558.665709,1288,37.888199,4.897685,-2.289256,-27.52031,20.300124,Hammer W Hedges


In [40]:
# import pandas as pd

# df = pd.DataFrame(tb_with_hedge)

# cols = df.columns.tolist()

# # Move 'Cumulative ROI%', 'Max Cumulative ROI%' and 'DD' to the last
# desired_cols = ['Cumulative ROI%', 'Max Cumulative ROI%', 'DD']
# cols = [col for col in cols if col not in desired_cols] + desired_cols
# # Reassign the columns to the DataFrame
# df = df[cols]

# df


In [41]:
tb = tb_with_hedge
tb['Cumulative ROI%'] = tb['Total ROI%'].cumsum()
tb['Max Cumulative ROI%'] = tb['Cumulative ROI%'].cummax()  # Maximum value so far
tb['DD'] = tb['Cumulative ROI%'] - tb['Max Cumulative ROI%']  # Drawdown
tb

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME,Hedge Strike,Hedge Entry Price,Hedge Exit Price,Hedge Points,Hedge PnL,Hedge ROI%,Total ROI%,Cumulative ROI%,Max Cumulative ROI%,DD
0,2019-01-25,Friday,2019-01-31,6,10900,C,87.20,11:45:00,108.95,0.05,2019-01-31,15:25:00,32.868020,EOD exit,87.15,0.8725,919.540230,79335.632184,7.933563,2019,1,89.55,0.05,8.013793,125.287356,2019-01-25 11:45:00,11450,1.25,0.05,-1.20,-1103.448276,-0.110345,7.823218,7.823218,7.823218,0.000000
1,2019-01-28,Monday,2019-01-31,3,10750,P,103.15,14:25:00,142.00,142.00,2019-01-29,12:45:00,49.254844,SL hit,-38.85,2.4515,514.800515,-21262.033462,-2.126203,2019,1,145.00,94.90,0.424710,69.176319,2019-01-28 14:25:00,0,NaN,NaN,NaN,NaN,0.000000,-2.126203,5.697015,7.823218,-2.126203
2,2019-01-30,Wednesday,2019-01-31,1,10700,P,65.40,13:35:00,103.50,103.50,2019-01-30,14:45:00,33.180077,SL hit,-38.10,1.6890,524.934383,-20886.614173,-2.088661,2019,1,107.45,40.60,1.301837,70.209974,2019-01-30 13:35:00,0,NaN,NaN,NaN,NaN,0.000000,-2.088661,3.608354,7.823218,-4.214865
3,2019-01-30,Wednesday,2019-01-31,1,10600,P,20.65,15:05:00,45.10,0.05,2019-01-31,15:25:00,48.433243,EOD exit,20.60,0.2070,817.995910,16681.390593,1.668139,2019,1,27.95,0.05,1.685072,108.384458,2019-01-30 15:05:00,10050,0.05,0.05,0.00,0.000000,0.000000,1.668139,5.276493,7.823218,-2.546726
4,2019-02-11,Monday,2019-02-14,3,10850,P,44.00,12:05:00,59.50,59.50,2019-02-13,15:05:00,48.848485,SL hit,-15.50,1.0350,1290.322581,-21335.483871,-2.133548,2019,2,60.75,20.35,3.051613,175.000000,2019-02-11 12:05:00,10400,1.20,0.35,-0.85,-1096.774194,-0.109677,-2.243226,3.033267,7.823218,-4.789952
5,2019-02-11,Monday,2019-02-14,3,10900,C,57.00,14:45:00,96.10,0.05,2019-02-14,15:25:00,48.808389,EOD exit,56.95,0.5705,511.508951,28838.618926,2.883862,2019,2,62.10,0.05,2.913043,69.693095,2019-02-11 14:45:00,0,NaN,NaN,NaN,NaN,0.000000,2.883862,5.917129,7.823218,-1.906090
6,2019-02-15,Friday,2019-02-21,6,10800,C,54.55,09:15:00,73.00,0.05,2019-02-21,15:25:00,42.433460,EOD exit,54.50,0.5460,1084.010840,58486.720867,5.848672,2019,2,55.85,0.05,5.907859,146.341463,2019-02-15 09:15:00,11350,1.20,0.20,-1.00,-1084.010840,-0.108401,5.740271,11.657400,11.657400,0.000000
7,2019-02-15,Friday,2019-02-21,6,10750,P,120.90,13:35:00,157.20,157.20,2019-02-19,15:05:00,37.351086,SL hit,-36.30,2.7810,550.964187,-21532.231405,-2.153223,2019,2,158.85,55.95,3.578512,74.035813,2019-02-15 13:35:00,0,NaN,NaN,NaN,NaN,0.000000,-2.153223,9.504177,11.657400,-2.153223
8,2019-02-20,Wednesday,2019-02-21,1,10650,P,29.80,14:25:00,59.35,0.05,2019-02-21,15:25:00,47.140523,EOD exit,29.75,0.2985,676.818951,19933.333333,1.993333,2019,2,31.20,0.05,2.013536,90.101523,2019-02-20 14:25:00,0,NaN,NaN,NaN,NaN,0.000000,1.993333,11.497510,11.657400,-0.159890
9,2019-02-22,Friday,2019-02-28,6,10800,C,73.35,10:15:00,77.65,77.65,2019-02-22,14:15:00,40.404798,SL hit,-4.30,1.5100,1851.851852,-10759.259259,-1.075926,2019,2,79.50,63.80,1.768519,250.000000,2019-02-22 10:15:00,11100,3.80,3.35,-0.45,-833.333333,-0.083333,-1.159259,10.338251,11.657400,-1.319149


In [42]:
# tb = tb.drop(columns=['RSI on Entry', 'day'])

In [43]:
tb.to_csv('Hammer_w_hedges.csv', index=False)

In [44]:
import pandas as pd

def calculate_dte_stats(df):
    """
    Calculate stats grouped by DTE: 
    - Returns Sum for each DTE
    - Max Drawdown (DD) for each DTE
    - Ratio of Returns Sum to Max DD for each DTE

    Parameters:
        df (pd.DataFrame): DataFrame containing a 'DTE' column, 'returns', and 'drawdown' columns.
    
    Returns:
        pd.DataFrame: DataFrame with aggregated statistics by DTE.
    """
    # print(df.columns)
    if 'DTE' not in df.columns or 'Total ROI%' not in df.columns or 'DD' not in df.columns:
        raise ValueError("DataFrame must contain 'DTE', 'returns', and 'drawdown' columns.")
    
    # Group by DTE and calculate required stats
    grouped_stats = df.groupby('DTE').agg(
        returns_sum=('Total ROI%', 'sum'),
        max_drawdown=('DD', 'min'),  # Assuming 'drawdown' contains negative values
        total_trades=('DTE', 'count')
    ).reset_index()
    
    # Calculate ratio of returns sum to max drawdown
    grouped_stats['returns_to_max_dd_ratio'] = (
        grouped_stats['returns_sum'] / grouped_stats['max_drawdown'].abs()
    )

    return grouped_stats


In [46]:
stats_dte = calculate_dte_stats(tb)
stats_dte

,DTE,returns_sum,max_drawdown,total_trades,returns_to_max_dd_ratio
0,1,57.980817,-24.076547,137,2.408187
1,2,248.155551,-27.389567,221,9.060222
2,3,89.776089,-27.520310,315,3.262176
3,5,-4.266995,-17.519295,22,-0.243560
4,6,167.020247,-25.440187,593,6.565213
